In [1]:
%%time
# Program streamlines orders

# STATUS: Complete
# Runtime: 250 ms

import pandas as pd
import numpy as np

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
horizon = 50
base_put = 0.99

# Read the main file
df_puts = pd.read_pickle(datapath+'df_main.pkl')

# Compute price variance percentage
df_puts['price_var'] = (df_puts.pBidPrice - df_puts.pLTP)
df_puts['price_var_pct'] = (df_puts.pBidPrice - df_puts.pLTP)/df_puts.pBidPrice

# Establish target price and substitute NaN with bidprice
df_puts['tgt_price'] = np.where(df_puts.price_var < -10, df_puts.pBidPrice+0.15, df_puts.pLTP)
df_puts.loc[df_puts.tgt_price.isna(), 'tgt_price'] = df_puts.pBidPrice + 0.15

# Determine return on margin
df_puts['RoM'] = df_puts.tgt_price/df_puts.marginpct/df_puts.undPrice*252/df_puts.DTE*100

# Filter the puts with some RoM
df_puts = df_puts.loc[(df_puts.Type == 'P') & 
                      (df_puts.RoM > 0), :].sort_values(['ibSymbol', 
                                                         'DTE', 'RoM'], ascending=False)

cols = ['ibSymbol', 'undPrice', 'Strike', 'Expiry', 'DTE', 
        'pLTP', 'pBidPrice', 'tgt_price', 'Type', 'Mlot', 'Delta', 'RoM']

# Screen timeframe within horizon
df_puts1 = df_puts[(df_puts.DTE.between(11, horizon, inclusive=False))]

# Keep only high delta records
df_puts2 = df_puts1[df_puts1.Delta >= base_put]

# Sort for RoM
df_puts2 = df_puts2.sort_values(['RoM'], ascending=False)

# Pickle the puts
df_puts2.to_pickle(datapath+'df_preorder_puts.pkl')

# make csv for lower-limit of options
df_th = pd.DataFrame(df_puts2[cols].sort_values(['RoM'], 
                    ascending=False).ibSymbol.unique())
df_th['Des'] = 'DES'   
df_th['Market'] = 'NSE'

df_th = df_th[['Des', 0, 'Market']].rename(\
    columns={0:'ibSymbol'}).sort_values(['ibSymbol'], ascending=True)

# Read the index pickle for stock / index differentiation
indexes = list(pd.read_pickle(datapath+'df_nse_idx_symbols.pkl').ibSymbol)
type_series = ['IND' if d in indexes else 'STK' for d in df_th.ibSymbol]
df_th['Type'] = type_series

# Get the min of close from ohlc for these symbols and merge
df_ohlc = pd.read_pickle(datapath+'df_ohlc.pkl')
df_min = pd.DataFrame(df_ohlc.groupby('ibSymbol').
                      C.min()).reset_index().rename(columns={'C': 'minC'})
df_th = df_th.merge(df_min)

# Write the threshold csv
cols = ['Des', 'ibSymbol', 'Type', 'Market', 'minC']
df_th[cols].to_csv(datapath+'put_threshold.csv', index=False, header=0)

Wall time: 1.01 s
